# 观察者的偏差

## 红线问题

在马萨诸塞州，“红线”是连接剑桥和波士顿的地铁线路。上下班高峰期，红线列车平均7~8分钟运行一趟。当到达车站时，我可以根据站台上的乘客人数估算下一班车到达的时间。如果只有几个人，就推测刚刚错过了地铁，下一班地铁预计要等约7分钟。如果站台上有较多乘客，就估计地铁会很快到达。但如果有相当多的乘客，则要怀疑列车未能如期运行，所以会回到街上叫出租车出行。

随机到达的乘客所看到的列车间隔的平均值，比实际的平均值要高一些，因为乘客到达的时间间隔可能是一个较大的区间。  
考虑一个简单的例子，假设列车间隔是5或10分钟（等概率），则列车平均时间7.5分钟，乘客观察到的列车间隔平均值是8.33分钟。  

这种**观察者偏差**在许多情况下出现，学生认为班级比实际的要大是因为他们经常上大课，飞机上的乘客认为飞机比实际更满是因为他们常常乘坐满员的航班。在每种情况下，实际分布中的值都按照比例被**过采样**了。

In [1]:
def BiasPmf(pmf):
    # pmf是实际的分布，new_pmf是偏分布
    new_pmf = pmf.Copy()
    for x, p in pmf.Items():
        new_pmf.Mult(x, x)
    new_pmf.Normalize()
    return new_pmf

## 等待时间
给定zb的分布，我们可以计算出等待时间y的分布，是根据每一个间隔的概率加权了的均匀分布的混合分布。

In [2]:
from thinkbayes import Pmf, MakeUniformPmf, MakeMixture
def PmfOfWaitTime(pmf_zb):
    metapmf = Pmf()
    for gap, prob in pmf_zb.Items():
        uniform = MakeUniformPmf(0, gap)
        metapmf.Set(uniform, prob)
    pmf_y = MakeMixture(metapmf)
    return pmf_y

- pmf_z 是 z 的非偏差分布
- pmf_zb 是乘客看到的列车间隔的偏差分布
- pmf_y 是等待时间的分布
- pmf_x 是经过的时间的分布，它和等待时间分布是一样的

In [3]:
class WaitTimeCalculator(object):
    def __init__(self, pmf_z):
        self.pmf_z = pmf_z
        self.pmf_zb = BiasPmf(pmf)
        
        self.pmf_y = PmfOfWaitTime(self.pmf_zb)
        self.pmf_x = self.pmf_y

z 的平均值为 7.8 分钟，zb 的平均值为 8.8 分钟，高出 z 约13%， y 均值为 4.4 分钟，是 zb 均值的一半。

## 预测等待时间
1. 利用列车间隔时间 $z$ 的分布来计算对于乘客而言列车经过时间 $x$
2. 然后，我们可以使用乘客到达率 $\lambda$ 和乘客数量 $k$ 来估计 $x$ 的分布
3. 最后，我们使用关系 $y = zp - x$ 可得 $y$ 的分布

In [4]:
class ElapsedTimeEstimator(object):
    def __init__(self, wtc, lam=2.0/60, num_passengers=15):
        # 经过时间的先验分布，pmf_x列车经过时间
        self.prior_x = Elapsed(wtc.pmf_x)
        # 经过时间的后验分布
        self.post_x = self.prior_x.Copy()
        self.post_x.Update((lam, num_passengers))
        # 等待时间的预测分布
        self.pmf_y = PredictWaitTime(wtc.pmf_zb, self.post_x)

In [5]:
from thinkbayes import Suite, EvalPoissonPmf
class Elapsed(Suite):
    def Likelihood(self, data, hypo):
        x = hypo
        lam, k = data
        # 给定到达率lam下，x时间内k位乘客到达的概率
        like = EvalPoissonPmf(k, lam*x)
        return like
    
def PredictWaitTime(pmf_zb, pmf_x):
    # pmf_zb乘客看到列车间隔的有偏分布，pmf_x列车经过时间的后验分布
    pmf_y = pmf_zb - pmf_x
    RemoveNegatives(pmf_y)
    return pmf_y

def RemoveNegatives(pmf):
    for val in pmf.Values():
        if val < 0:
            pmf.Remove(val)
    pmf.Normalize()

## 估计到达率
记录自己的等待时间以及在你等待期间新到站的乘客数量。
- 假设为 $\lambda$ 的取值
- $y$ 是你等待的时间
- $k$ 为等待期间到达的乘客人数

In [6]:
class ArrivalRate(Suite):
    # 已知lam的条件下，一段时间里遇到k个到达乘客的可能性
    def Likelihood(self, data, hypo):
        lam = hypo
        y, k = data
        like = EvalPoissonPmf(k, lam*y)
        return like

In [7]:
import numpy as np
class ArrivalRateEstimator(object):
    def __init__(self, passenger_data):
        # 假设每分钟到达0-5个乘客
        low, high = 0, 5
        n = 51
        hypos = np.linspace(low, high, n) / 60
        
        self.prior_lam = ArrivalRate(hypos)
        self.post_lam = self.prior_lam.Copy()
        for k1, y, k2 in passenger_data:
            self.post_lam.Update((y, k2))

## 消除不确定性
无论何时，分析中总有一些输入量带来的不确定性，我们可以通过下面这个步骤将这一因素考虑进来：
1. 实现基于不确定参数的确定值分析（在本例中是 $\lambda$ ）
2. 计算不确定参数的分布
3. 对参数的每个值进行分析，并生成一组预测分布
4. 使用参数分布所对应的权值计算出预测分布的混合分布

In [9]:
class WaitMixtureEstimator(object):
    def __init__(self, wtc, are, num_passengers=15):
        # are是包含了lam分布的ArrivalTimeEstimator实例
        self.metapmf = Pmf()
        for lam, prob in sorted(are.post_lam.Items()):
            ete = ElapsedTimeEstimator(wtc, lam, num_passengers)
            self.metapmf.Set(ete.pmf_y, prob)
        self.mixture = MakeMixture(self.metapmf)

## 决策分析